In [1]:
!pip install langfuse-haystack

import os
from getpass import getpass

from haystack import Pipeline
from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.connectors.langfuse import LangfuseConnector

Defaulting to user installation because normal site-packages is not writeable
  Using cached langfuse_haystack-0.0.4-py3-none-any.whl.metadata (4.7 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
Using cached langfuse_haystack-0.0.4-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 3.5 MB/s eta 0:00:004.0 MB/s eta 0:00:01
Using cached idna-3.7-py3-none-any.whl (66 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6


In [2]:
os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
os.environ["LANGFUSE_SECRET_KEY"] = getpass("Enter LANGFUSE_SECRET_KEY key:")
os.environ["LANGFUSE_PUBLIC_KEY"] = getpass("Enter LANGFUSE_PUBLIC_KEY key:")
os.environ["HAYSTACK_CONTENT_TRACING_ENABLED"] = "True"

In [3]:
pipe = Pipeline()
pipe.add_component("tracer", LangfuseConnector("Chat example"))
pipe.add_component("prompt_builder", DynamicChatPromptBuilder())
pipe.add_component("llm", OpenAIChatGenerator(model="gpt-3.5-turbo"))

pipe.connect("prompt_builder.prompt", "llm.messages")
messages = [
    ChatMessage.from_system(
        "Always respond in German even if some input data is in other languages."
    ),
    ChatMessage.from_user("Tell me about {{location}}"),
]

response = pipe.run(
    data={
        "prompt_builder": {
            "template_variables": {"location": "Berlin"},
            "prompt_source": messages,
        }
    }
)
print(response["llm"]["replies"][0])
print(response["tracer"]["trace_url"])

/Users/apple/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ChatMessage(content='Berlin ist die Hauptstadt und größte Stadt Deutschlands. Es ist bekannt für seine reiche Geschichte, lebendige Kunstszene, alternative Kultur und pulsierendes Nachtleben. Zu den bekanntesten Sehenswürdigkeiten gehören das Brandenburger Tor, der Berliner Dom, die Berliner Mauer und der Reichstag. Berlin ist auch ein wichtiger Standort für Technologie, Politik, Medien und Wissenschaft. Die Stadt hat eine vielfältige Bevölkerung und zieht Besucher aus der ganzen Welt an.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 122, 'prompt_tokens': 29, 'total_tokens': 151}})
https://cloud.langfuse.com/trace/aa71e3db-e2f7-4b37-950a-2a9cbcf5d9e1
